In [1]:
import numpy as np
import os

建立数据

In [2]:
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

使用np的svd分解来还原PCA降维

In [3]:
X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)#奇异矩阵分解
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]

In [6]:
s#奇异值

array([6.77645005, 2.82403671, 0.78116597])

In [7]:
m, n = X.shape

S = np.zeros(X_centered.shape)
S[:n, :n] = np.diag(s)#建立奇异值的对角阵

In [8]:
S

array([[6.77645005, 0.        , 0.        ],
       [0.        , 2.82403671, 0.        ],
       [0.        , 0.        , 0.78116597],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.

In [9]:
np.allclose(X_centered, U.dot(S).dot(Vt))#说明X由USV分解得到

True

In [10]:
W2 = Vt.T[:, :2]
X2D = X_centered.dot(W2)#将原始矩阵和奇异矩阵相乘得到转化后的二维矩阵

In [13]:
Vt.T[:, :2]

array([[ 0.93636116, -0.34027485],
       [ 0.29854881,  0.90119108],
       [ 0.18465208,  0.2684542 ]])

In [15]:
Vt.T[:, :2].shape

(3, 2)

In [17]:
X_centered.shape

(60, 3)

In [18]:
X2D_using_svd = X2D

PCA自动帮你进行中心化，所以不需要减去均值

In [19]:

from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

In [20]:
X2D[:5]

array([[ 1.26203346,  0.42067648],
       [-0.08001485, -0.35272239],
       [ 1.17545763,  0.36085729],
       [ 0.89305601, -0.30862856],
       [ 0.73016287, -0.25404049]])

In [21]:
X2D_using_svd[:5]

array([[-1.26203346, -0.42067648],
       [ 0.08001485,  0.35272239],
       [-1.17545763, -0.36085729],
       [-0.89305601,  0.30862856],
       [-0.73016287,  0.25404049]])

可以看得出，结果除了符号一模一样。

In [22]:
np.allclose(X2D, -X2D_using_svd)

True

In [25]:
X3D_inv = pca.inverse_transform(X2D)#可以把2维还原到3维，但是会有一定的信息损失

In [24]:
np.allclose(X3D_inv, X)

False

损失非常小

In [26]:
np.mean(np.sum(np.square(X3D_inv - X), axis=1))

0.010170337792848549

其实这个转化就是反过来乘了转化矩阵的转置

In [27]:
X3D_inv_using_svd = X2D_using_svd.dot(Vt[:2, :])

In [30]:
np.allclose(X3D_inv_using_svd, X3D_inv - pca.mean_)#sklearn的pca会帮你恢复去中心化。

True

In [33]:
Vt[:2, :]

array([[ 0.93636116,  0.29854881,  0.18465208],
       [-0.34027485,  0.90119108,  0.2684542 ]])

In [32]:
pca.components_

array([[-0.93636116, -0.29854881, -0.18465208],
       [ 0.34027485, -0.90119108, -0.2684542 ]])

可以看得出主成分就是VT

In [34]:
#可以看一下解释百分比，第一维解释了84%
pca.explained_variance_ratio_

array([0.84248607, 0.14631839])

In [35]:
#其实就是奇异值的平方/奇异值的平方和
np.square(s) / np.square(s).sum()

array([0.84248607, 0.14631839, 0.01119554])

In [37]:
1 - pca.explained_variance_ratio_.sum()

0.011195535570688975

In [44]:
#使用累加可以看到在第几维达到了多少的解释率
np.cumsum(pca.explained_variance_ratio_) 

array([0.84248607, 0.98880446])

In [45]:
#也可以直接制定0.95解释率
pca = PCA(n_components=0.95) 
X_reduced = pca.fit_transform(X)

In [ ]:
#当维度过大，样本数量过多，并且样本数量于特征数量80%的时候会使用随机PCA，降低复杂度。
rnd_pca = PCA(n_components=2 svd_solver="randomized") X_reduced = rnd_pca.fit_transform(X_train)

In [ ]:
#如果数据集很大可以用增量学习
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train, n_batches):
    print(".", end="") # not shown in the book
    inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X_train)

memmap它允许将大文件分成小段进行读写，而不是一次性将整个数组读入内存
等于把这个文件按段保存磁盘里，再分段读入。

In [ ]:
filename = "my_mnist.data"
m, n = X_train.shape

X_mm = np.memmap(filename, dtype='float32', mode='write', shape=(m, n))
X_mm[:] = X_train

In [ ]:
del X_mm

In [ ]:
X_mm = np.memmap(filename, dtype="float32", mode="readonly", shape=(m, n))

batch_size = m // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)

在做类别可视化分析的时候经常用Tsne来降维，但是直接对大数据集用TSNE会很慢，这里推荐先用PCA降维之后再用TSNE可以显著提高速度
因为原理不同，导致，tsne 保留下的属性信息，更具代表性，也即最能体现样本间的差异；



In [ ]:
我认为在对欺诈客群进行社群划分之后来可视化可以看划分的效果好不好。

In [ ]:
pca_tsne = Pipeline([
    ("pca", PCA(n_components=0.95, random_state=42)),
    ("tsne", TSNE(n_components=2, random_state=42)),
])
t0 = time.time()
X_pca_tsne_reduced = pca_tsne.fit_transform(X)
t1 = time.time()
print("PCA+t-SNE took {:.1f}s.".format(t1 - t0))

In [ ]:
plt.figure(figsize=(13,10))
plt.scatter(X_pca_tsne_reduced[:, 0], X_pca_tsne_reduced[:, 1], c=y, cmap="jet")#要放入类别标签 y
plt.axis('off')
plt.colorbar()
plt.show()